In [27]:

# !pip install llama_index.vector_stores.qdrant
# !pip install llama_index.core
# !pip install llama_index
# !pip install llama_index.embeddings.huggingface


In [17]:
import nest_asyncio
nest_asyncio.apply()

import qdrant_client
from qdrant_client.models import VectorParams, Distance
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, ServiceContext, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

# Define collection name
collection_name = "chat_with_docs"

# Connect to Qdrant
try:
    client = qdrant_client.QdrantClient(
        host="host.docker.internal",
        port=6333
    )
    # Test connection by getting collection info
    try:
        client.get_collection(collection_name)
        print(f"Successfully connected to Qdrant and found collection '{collection_name}'")
    except Exception as e:
        print(f"Collection '{collection_name}' not found. Creating it now...")
        # Get the dimension from your embedding model
        embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5",
                                          trust_remote_code=True)
        embedding_dimension = 1024  # bge-large-en-v1.5 uses 1024 dimensions
        
        # Create the collection with proper format using VectorParams
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(
                size=embedding_dimension,
                distance=Distance.COSINE
            )
        )
        print(f"Created collection '{collection_name}'")
except Exception as e:
    print(f"Error connecting to Qdrant: {e}")
    print("\nPossible solutions:")
    print("1. Make sure Qdrant is running locally with: docker run -p 6333:6333 qdrant/qdrant")
    print("2. Check if port 6333 is not blocked by firewall")
    print("3. Verify no other application is using port 6333")
    raise

# Load documents
input_dir_path = './docs'
loader = SimpleDirectoryReader(
    input_dir=input_dir_path,
    required_exts=[".pdf"],
    recursive=True
)
docs = loader.load_data()
print(f"Loaded {len(docs)} documents")

# Set up embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5",
                                  trust_remote_code=True)
Settings.embed_model = embed_model

# Create index function
def create_index(documents):
    print("Creating vector store...")
    vector_store = QdrantVectorStore(client=client,
                                    collection_name=collection_name)
    
    print("Setting up storage context...")
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    print("Building index from documents (this may take a while)...")
    index = VectorStoreIndex.from_documents(documents,
                                           storage_context=storage_context)
    
    print("Index creation complete!")
    return index

# Create the index
try:
    index = create_index(docs)
    print("Successfully created index!")
except Exception as e:
    print(f"Error creating index: {e}")
    raise

Successfully connected to Qdrant and found collection 'chat_with_docs'
Loaded 32 documents
Creating vector store...
Setting up storage context...
Building index from documents (this may take a while)...
Index creation complete!
Successfully created index!


In [18]:
# !pip install requests

# !pip install llama_index.llms.ollama


In [20]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

llm = Ollama(model="llama3.2:1b", request_timeout=120.0)

Settings.llm = llm

In [21]:
from llama_index.core import PromptTemplate

template = """Context information is below:
              ---------------------
              {context_str}
              ---------------------
              Given the context information above I want you to think
              step by step to answer the query in a crisp manner,
              incase you don't know the answer say 'I don't know!'
            
              Query: {query_str}
        
              Answer:"""

qa_prompt_tmpl = PromptTemplate(template)

In [14]:
from llama_index.core.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-2-v2", 
    top_n=3
)

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/62.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [24]:
query_engine = index.as_query_engine(similarity_top_k=10,
                                     node_postprocessors=[rerank])

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

response = query_engine.query("What exactly is DSPy?")

In [25]:
from IPython.display import Markdown, display

display(Markdown(str(response)))

DSPy stands for Deep Speech Processing with Yaml, which is a library and framework for natural language processing (NLP) in Python. It provides an easy-to-use interface for implementing various NLP tasks, including text processing, question answering, and more.

In the context of this preprint, DSPy is being used to develop and train AI models for natural language understanding and generation. The library offers a range of modules and interfaces that allow developers to create custom NLP models, train them on large datasets, and fine-tune their performance using various optimization techniques.

Some specific features of DSPy include:

* Natural Language Signatures: DSPy allows users to define natural language signatures, which are concise and declarative specifications for how a model should be prompted or asked questions.
* Module-based Programming: DSPy provides an interface for creating custom modules that can be used as building blocks for more complex models. These modules can be easily composed together to create more sophisticated models.
* Bootstrapping: DSPy offers several bootstrapping techniques, including simple fine-tuning and multi-hop training, which allow developers to optimize their model's performance on specific tasks or datasets.

The preprint also discusses two key components of DSPy: Predict and ChainOfThought. The Predict module is a basic building block that allows developers to define natural language signatures and train models using them. The ChainOfThought module extends the Predict module by adding additional logic for generating queries and responses, which can be used to create more complex models.

Overall, DSPy provides a powerful and flexible framework for developing NLP models in Python, allowing users to easily define and implement their own natural language processing tasks.